In [ ]:
import pandas as pd
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
import numpy as np

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline

plt.rcParams['figure.figsize'] = (9, 6)
sns.set(context='notebook', style='whitegrid', font_scale=1.2)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
import statsmodels.api as sm

In [ ]:
# Make better use of Jupyter Notebook cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
train_identity = pd.read_csv('/Users/arcarter/Git_Repos/project_03/data/ieee-fraud-detection/train_identity.csv')

In [ ]:
train_identity.info()

In [ ]:
#train_identity.sample(15)

In [ ]:
train_transaction = pd.read_csv('/Users/arcarter/Git_Repos/project_03/data/ieee-fraud-detection/train_transaction.csv')

In [ ]:
train_transaction.info()

In [ ]:
#train_transaction.head(25)

In [ ]:
train_transaction_identity = train_transaction.merge(train_identity, on='TransactionID',how='left')

In [ ]:
train_transaction_identity.shape

In [ ]:
train_transaction_identity.info()

In [ ]:
def df_column_unique_values(df, top_n = 5):
    for col_name, values in df.iteritems():
        col_value_counts = values.value_counts()
        print(f"{col_name} : {len(col_value_counts)}")
        col_value_count_list = [
            "'" + str(c) + "'" + ":" + str(n) for c, n in sorted(
                col_value_counts.items(),
                key=lambda kv: kv[1],
                reverse=True
            )
        ]
        print(", ".join(col_value_count_list[:min(len(col_value_count_list), top_n)]))
        # print ('\\n')

In [ ]:
df_column_unique_values(train_transaction_identity)

In [ ]:
dataframe_dtypes_orig = pd.DataFrame(train_transaction_identity.dtypes)
dataframe_dtypes_orig.head(200)

In [ ]:
#test_identity = pd.read_csv('/Users/arcarter/Git_Repos/project_03/data/ieee-fraud-detection/test_identity.csv')

In [ ]:
#test_transaction = pd.read_csv('/Users/arcarter/Git_Repos/project_03/data/ieee-fraud-detection/test_transaction.csv')

In [ ]:
#test_transaction_identity = test_transaction.merge(test_identity, on='TransactionID',how='left')

In [ ]:

plt.figure(figsize=(2.5,5)) 
plt.title("Fraud Transaction Distribution") 
p1 = sns.countplot(train_transaction_identity['isFraud']) 
for p in p1.patches: p1.annotate('{:6.2f}%'.format(p.get_height()/len(train_transaction_identity)*100), (p.get_x()+0.1, p.get_height()+50)) 
plt.show()

In [ ]:

plt.figure(figsize=(2.5,5)) 
plt.title("Fraud Transaction Distribution") 
p1 = sns.countplot(train_transaction_identity['isFraud']) 
for p in p1.patches: p1.annotate('{:6.2f}%'.format(p.get_height()/len(train_transaction_identity)*100), (p.get_x()+0.1, p.get_height()+50)) 
plt.bar(labels, values, color=['blue', 'orange'])
plt.show()


In [ ]:
target_count = train_transaction_identity['isFraud'].value_counts()

In [ ]:
target_count

In [ ]:
####################
####################
#
# Remove outlier Transaction Amounts
#
####################
####################

In [ ]:
train_transaction_identity_noOutlier = train_transaction_identity[train_transaction_identity['TransactionAmt']<10000]

In [ ]:
train_transaction_identity_noOutlier.info()

In [ ]:
### Remove high NAN columns
df_column_completion = pd.DataFrame((np.sum(pd.isnull(train_transaction_identity_noOutlier)).
                   sort_values(ascending=False)/len(train_transaction_identity_noOutlier))*100)

df_column_completion.rename(columns = {0:'% Null'}, inplace = True)

In [ ]:
df_column_completion.head(20)
                         

In [ ]:
train_transaction_identity_noOutlier_limNAN = train_transaction_identity_noOutlier.copy()

In [ ]:
#Remove variables with missing values more than 30 percent

A = (np.sum(pd.isnull(train_transaction_identity_noOutlier_limNAN)).sort_values(ascending=False)/len(train_transaction_identity_noOutlier_limNAN))*100
Removed_col = A[A>30].index
print(Removed_col)


In [ ]:
train_transaction_identity_noOutlier_limNAN.drop(columns=Removed_col, inplace=True)

In [ ]:
train_transaction_identity_noOutlier_limNAN.head()

In [ ]:
################
# One Hot Encode
################

In [ ]:
train_transaction_identity_noOutlier_limNAN["card6"].fillna('None', inplace=True)

In [ ]:
cat_X = train_transaction_identity_noOutlier_limNAN.loc[:, ['card6']]

In [ ]:
ohe = OneHotEncoder(sparse=False)

ohe.fit(cat_X)

ohe_X = ohe.transform(cat_X)

columns = ohe.get_feature_names(['card6'])

ohe_X_df = pd.DataFrame(ohe_X, columns=columns, index=cat_X.index)

ohe_X_df.head()

In [ ]:
train_transaction_identity_noOutlier_limNAN["card4"].fillna('None', inplace=True)

In [ ]:
cat_X2 = train_transaction_identity_noOutlier_limNAN.loc[:, ['card4']]

In [ ]:
ohe = OneHotEncoder(sparse=False)

ohe.fit(cat_X2)

ohe_X2 = ohe.transform(cat_X2)

columns = ohe.get_feature_names(['card4'])

ohe_X2_df = pd.DataFrame(ohe_X2, columns=columns, index=cat_X2.index)

ohe_X2_df.head()

In [ ]:
combined_df = pd.concat([ohe_X_df, ohe_X2_df,train_transaction_identity_noOutlier_limNAN], axis=1)
combined_df.head()

In [ ]:
train_transaction_identity_noOutlier_limNAN["P_emaildomain"].dtype

In [ ]:
train_transaction_identity_noOutlier_limNAN["P_emaildomain"].fillna('None', inplace=True)

In [ ]:
train_transaction_identity_noOutlier_limNAN['P_emaildomain'].value_counts()

In [ ]:
train_transaction_identity_noOutlier_limNAN['P_emaildomain_gmail'] = np.where(train_transaction_identity_noOutlier_limNAN['P_emaildomain']== 'gmail.com', 1, 0)

In [ ]:
train_transaction_identity_noOutlier_limNAN['P_emaildomain_yahoo'] = np.where(train_transaction_identity_noOutlier_limNAN['P_emaildomain']== 'yahoo.com', 1, 0)

In [ ]:
train_transaction_identity_noOutlier_limNAN['P_emaildomain_none'] = np.where(train_transaction_identity_noOutlier_limNAN['P_emaildomain']== 'none', 1, 0)

In [ ]:
#train_transaction_identity_noOutlier_limNAN['P_emaildomain_small'] = np.where(train_transaction_identity_noOutlier_limNAN[(train_transaction_identity_noOutlier_limNAN['P_emaildomain']!='gmail.com') | (train_transaction_identity_noOutlier_limNAN['P_emaildomain']!='yahoo.com')],1,0)

#result = result[(result['var']>0.25) | (result['var']<-0.25)]

In [ ]:
del train_transaction_identity_noOutlier_limNAN['P_emaildomain_MainEmails']

In [ ]:
train_transaction_identity_noOutlier_limNAN['ProductCD'].value_counts()

In [ ]:
train_transaction_identity_noOutlier_limNAN["ProductCD"].fillna('None', inplace=True)

In [ ]:
cat_X3 = train_transaction_identity_noOutlier_limNAN.loc[:, ['ProductCD']]

In [ ]:
ohe = OneHotEncoder(sparse=False)

ohe.fit(cat_X3)

ohe_X3 = ohe.transform(cat_X3)

columns = ohe.get_feature_names(['ProductCD'])

ohe_X3_df = pd.DataFrame(ohe_X3, columns=columns, index=cat_X3.index)

ohe_X3_df.head()

In [ ]:
combined_df.info()

In [ ]:
combined_df = pd.concat([ohe_X_df, ohe_X2_df,ohe_X3_df,train_transaction_identity_noOutlier_limNAN], axis=1)
combined_df.head()

In [ ]:
combined_df_final = combined_df.copy()

In [ ]:
del combined_df_final['card6']

In [ ]:
del combined_df_final['card4']
del combined_df_final['P_emaildomain']
del combined_df_final['ProductCD']

In [ ]:
combined_df_final['hour'] = (combined_df_final['TransactionDT']//(3600))%24

In [ ]:
train_hour = (combined_df_final.groupby(['isFraud'])['hour']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('hour'))

In [ ]:
sns.barplot(x="hour", y="percentage", hue="isFraud", data=train_hour)

In [ ]:
combined_df_final.tail()

In [ ]:
combined_df_final.isFraud.dtype

In [ ]:
####################
####################
#
# Final prep of data and Train Validate Split
#
####################
####################

In [ ]:
combined_df_final.loc[:, 'isFraud'] = combined_df_final['isFraud'].astype('category')

In [ ]:
df_train, df_test = train_test_split(combined_df_final,
                                     test_size=0.2, 
                                     random_state=42)

In [ ]:
print(df_train.shape)
print(df_test.shape)

In [ ]:
#sns.pairplot(df_train.sample(10000), corner=True, height=3, plot_kws={'size': 5}, hue='isFraud');

In [ ]:
sns.violinplot(x='price_per_sqft',
               y='in_sf',
               data=df_train);

In [ ]:
####################
####################
#
# Separability
#
####################
####################

In [ ]:
# `price_per_sqft` should work for our purposes
sns.violinplot(x='TransactionAmt',
               y='isFraud',
               data=df_train);

In [ ]:
import statsmodels.api as sm

In [ ]:
# For this first example, we'll employ statsmodels
lm_1 = sm.Logit(df_train['isFraud'],  # with statsmodels, `y` comes first
                sm.add_constant(df_train[['TransactionAmt']]))  # and then `x`
lm_1 = lm_1.fit()

In [ ]:
lm_1.summary()

In [ ]:
# Let's go down the sklearn path, since this is better used in model pipelines
from sklearn.linear_model import LogisticRegression

In [ ]:
# By default, LogisticRegression will fit an intercept, as we did with statsmodels
lm_1 = LogisticRegression(solver='newton-cg',  # For comparison, use the same solver as statsmodels default
                          C=100000)  # No regularization

lm_1.fit(df_train[['TransactionAmt']], df_train['isFraud'])

In [ ]:
print('intercept: ', round(lm_1.intercept_[0], 4))
print('TransactionAmt coef: ', round(lm_1.coef_[0][0], 4))

In [ ]:
df_eval = df_test.copy()
df_eval['pred'] = lm_1.predict(df_test[['TransactionAmt']])
df_eval.loc[:, 'pred'] = df_eval['pred'].astype('category')

In [ ]:
df_eval.loc[:, 'pred'] = df_eval['pred'].astype('int')

In [ ]:
df_eval.loc[:, 'isFraud'] = df_eval['isFraud'].astype('int')

In [ ]:
df_eval.isFraud.dtype

In [ ]:
df_eval.pred.dtype

In [ ]:
df_eval['correct_pred'] = df_eval['pred'] == df_eval['isFraud']

In [ ]:
df_eval.correct_pred.dtype

In [ ]:
sns.stripplot(data=df_eval.sample(20000),
              x='TransactionAmt',
              y='pred',
              hue='correct_pred',
              palette={False: '#f03b20', True: '#3182bd'});

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_formats = ['retina']  # or svg
%matplotlib inline

plt.rcParams['figure.figsize'] = (9, 6)
sns.set(context='notebook', style='whitegrid', font_scale=1.2)

from sklearn.datasets import fetch_california_housing
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
import xgboost as xgb

In [ ]:
combined_df_final_xg = combined_df_final.copy()

In [ ]:
#y = combined_df_final_xg['isFraud']
pd.to_numeric(y)

In [ ]:
combined_df_final_xg.shape

In [ ]:
combined_df_final_xg = combined_df_final_xg[combined_df_final_xg.columns.difference(['isFraud','M6'])]

In [ ]:
X = combined_df_final_xg[cols]

In [ ]:
X.head()

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
cols = ['P_emaildomain_gmail','P_emaildomain_none','P_emaildomain_yahoo','ProductCD_C','ProductCD_H','ProductCD_R','ProductCD_S','ProductCD_W','TransactionAmt','TransactionDT','card1','card4_None','card4_american express','card4_discover','card4_mastercard','card4_visa','card6_None','card6_charge card','card6_credit','card6_debit','card6_debit or credit','hour']

In [ ]:
X.dtypes

In [ ]:
y = pd.to_numeric(y)

In [ ]:
y.dtype


In [ ]:
X.isnull().sum()

In [ ]:
X["addr1"].fillna('None', inplace=True)
X["addr2"].fillna('None', inplace=True)
X["card2"].fillna('None', inplace=True)
X["card3"].fillna('None', inplace=True)
X["card5"].fillna('None', inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression



In [ ]:
X.shape

In [ ]:
#Split data into 3: 60% train, 20% validation, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2018)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=2019)

#Evaluate models with Root Mean Squared Error
def rmse(actuals, preds):
    return np.sqrt(((actuals - preds) ** 2).mean())

In [ ]:
X_train.head()

In [ ]:
import pickle

In [ ]:
filename 

In [ ]:


pickle.dump(X, open( "save.p", "wb" ) )

In [ ]:
pickle.dump(y, open( "save2.p", "wb" ) )

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)
rmse(lr.predict(X_test), y_test)

In [ ]:
rf = RandomForestRegressor(n_estimators = 100, max_features = 3, n_jobs=-1)
rf.fit(X_train,y_train)
rmse(rf.predict(X_test), y_test)

In [ ]:
rf_preds = rf.predict(X_test)

In [ ]:
rf_preds[:10]

In [ ]:
y_test[:10]

In [ ]:
rf.fit(X_train,y_train)

In [ ]:
97/3


In [ ]:
gbm = xgb.XGBRegressor( 
                       n_estimators=30000, #arbitrary large number
                       max_depth=3,
                       objective="reg:squarederror",
                       learning_rate=.1, 
                       subsample=1,
                       min_child_weight=1,
                       colsample_bytree=.8
                      )

eval_set=[(X_train,y_train),(X_val,y_val)] #tracking train/validation error as we go
fit_model = gbm.fit( 
                    X_train, y_train, 
                    eval_set=eval_set,
                    eval_metric='rmse',
                    early_stopping_rounds=20,
                    verbose=True #gives output log as below
                   )

In [ ]:
rmse(gbm.predict(X_test, ntree_limit=gbm.best_ntree_limit),y_test)

In [ ]:
preds = gbm.predict(X_test, ntree_limit=gbm.best_ntree_limit)

In [ ]:
# Print confusion matrix for kNN
RF_matrix = confusion_matrix(rf.predict(X_test), y_test)
plt.figure(dpi=150)
values_format = '.5f'
sns.heatmap(log_confusion, cmap=plt.cm.Blues,fmt = ".1f", annot=True, square=True,
           xticklabels=[0,1],
           yticklabels=[0,1])

plt.xlabel('Prediction Fraud vs. Not Fraud')
plt.ylabel('Actual Fraud vs. Not Fraud')
plt.title('Logistic Regression Confusion Matrix');

In [ ]:
rmse(gbm.predict(X_val, ntree_limit=gbm.best_ntree_limit),y_val) 

In [ ]:
xgb.plot_importance(gbm)
xgb.plot_importance(gbm, importance_type='gain')

In [ ]:
gbm.get_booster().get_score(importance_type='weight') #extract raw frequency scores

In [ ]:
def make_confusion_matrix(model, threshold=0.5):
    # Predict class 1 if probability of being in class 1 is greater than threshold
    # (model.predict(X_test) does this automatically with a threshold of 0.5)
    y_predict = (model.predict_proba(X_test)[:, 1] >= threshold)
    fraud_confusion = confusion_matrix(y_test, y_predict)
    plt.figure(dpi=80)
    sns.heatmap(fraud_confusion, cmap=plt.cm.Blues, annot=True, square=True, fmt='d',
           xticklabels=['legit', 'fraud'],
           yticklabels=['legit', 'fraud']);
    plt.xlabel('prediction')
    plt.ylabel('actual')



In [ ]:
make_confusion_matrix(gbm)

In [ ]:

plt.title("Card3") 
p1 = sns.countplot(train_transaction_identity['card3']) 

In [ ]:
df.hist(column='card3', bins=100)

In [ ]:
plt.figure(figsize=(2.5,5)) 
plt.title("Fraud Transaction Distribution") 
p1 = sns.countplot(train_transaction_identity['isFraud']) 
for p in p1.patches: p1.annotate('{:6.2f}%'.format(p.get_height()/len(train_transaction_identity)*100), (p.get_x()+0.1, p.get_height()+50)) 
plt.show()

In [ ]:
train_transaction_identity['day'] = (train_transaction_identity['TransactionDT']//(3600*24)-1)%7

In [ ]:
plt.figure(figsize=(11,6))
plt.subplot(1,2,1)
percentage = lambda i: len(i) / float(len(train_transaction_identity['day'])) * 100
ax = sns.barplot(x=train_transaction_identity['day'], y=train_transaction_identity['day'],  estimator=percentage)
ax.set(ylabel="Percent")
plt.title('Train')


In [ ]:
train_day = (train_transaction_identity.groupby(['isFraud'])['day']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('day'))

In [ ]:
sns.barplot(x="day", y="percentage", hue="isFraud", data=train_day)

In [ ]:
train_transaction_identity['hour'] = (train_transaction_identity['TransactionDT']//(3600))%24

In [ ]:
train_hour = (train_transaction_identity.groupby(['isFraud'])['hour']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('hour'))

In [ ]:
sns.barplot(x="hour", y="percentage", hue="isFraud", data=train_hour)

In [ ]:
train_card_type = (train_transaction_identity.groupby(['isFraud'])['card6']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index())

In [ ]:
sns.barplot(x="card6", y="percentage", hue="isFraud", data=train_card_type)

In [ ]:
logistic_test = train_transaction_identity[["isFraud", "hour","TransactionAmt","card6"]]

In [ ]:
logistic_test = logistic_test[logistic_test['TransactionAmt']<10000]

In [ ]:
logistic_test.shape

In [ ]:
logistic_test.info

In [ ]:
#sns.pairplot(logistic_test, corner=True, height=5, plot_kws={'size': 3}, hue='isFraud');

In [ ]:
sns.violinplot(y='hour',
               x='isFraud',
               data=logistic_test);

In [ ]:
df_train, df_validate = train_test_split(logistic_test,
                                     test_size=0.2, 
                                     random_state=42)

In [ ]:
df_train.shape

In [ ]:
df_validate.shape

In [ ]:
sns.pairplot(df_train.sample(10000), corner=True, height=3, plot_kws={'size': 5}, hue='isFraud');

In [ ]:
sns.scatterplot(data=logistic_test,
                x='hour',
                y='TransactionAmt',
                hue='isFraud');

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve
from sklearn.preprocessing import StandardScaler

In [ ]:
features = ['hour', 'TransactionAmt']

# Since we're using more than one feature, let's scale our features
scaler = StandardScaler()
X_train = scaler.fit_transform(df_train[features])
y_train = df_train['isFraud']

In [ ]:
lm_2 = LogisticRegression()  # We'll also regularize our features

In [ ]:
lm_2.fit(X_train, y_train)

In [ ]:
X_validate = scaler.transform(df_test[features])
preds = lm_2.predict(X_validate)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')
%matplotlib inline
np.set_printoptions(suppress=True)

In [ ]:
confusion_matrix(df_test['isFraud'], 
                 preds)

In [ ]:
confusion_matrix(df_validate['isFraud'], 
                 preds)

In [ ]:
# Print confusion matrix for kNN
log_confusion = confusion_matrix(df_test['isFraud'], preds)
plt.figure(dpi=150)
values_format = '.5f'
sns.heatmap(log_confusion, cmap=plt.cm.Blues,fmt = ".1f", annot=True, square=True,
           xticklabels=[0,1],
           yticklabels=[0,1])

plt.xlabel('Prediction Fraud vs. Not Fraud')
plt.ylabel('Actual Fraud vs. Not Fraud')
plt.title('Logistic Regression Confusion Matrix');

In [ ]:
confusion_matrix(df_validate['isFraud'], 
                 preds)

In [ ]:
def make_confusion_matrix(model, threshold=0.5):
    # Predict class 1 if probability of being in class 1 is greater than threshold
    # (model.predict(X_test) does this automatically with a threshold of 0.5)
    y_predict = (model.predict_proba(X_test)[:, 1] >= threshold)
    fraud_confusion = confusion_matrix(y_test, y_predict)
    plt.figure(dpi=80)
    sns.heatmap(fraud_confusion, cmap=plt.cm.Blues, annot=True, square=True, fmt='d',
           xticklabels=['legit', 'fraud'],
           yticklabels=['legit', 'fraud']);
    plt.xlabel('prediction')
    plt.ylabel('actual')

In [ ]:
xgb.plot_importance(gbm)
xgb.plot_importance(gbm, importance_type='gain')

In [ ]:
from xgboost import XGBClassifier
from sklearn.datasets import load_iris
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold

In [ ]:
xgbc = XGBClassifier()
print(xgbc)
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='multi:softprob', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1) 

In [ ]:
#Split data into 3: 60% train, 20% validation, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2018)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=2019)

#Evaluate models with Root Mean Squared Error
def rmse(actuals, preds):
    return np.sqrt(((actuals - preds) ** 2).mean())

In [ ]:
pickle.dump( X_train, open( "X_train.p", "wb" ) )

In [ ]:
pickle.dump( X_test, open( "X_test.p", "wb" ) )

In [ ]:
pickle.dump( y_train, open( "y_train.p", "wb" ) )

In [ ]:
pickle.dump( y_test, open( "y_test.p", "wb" ) )

In [ ]:
pickle.dump( X_val, open( "X_val", "wb" ) )

In [ ]:
pickle.dump( y_val, open( "y_val", "wb" ) )

In [ ]:
y_val[5]

In [ ]:
y_val[57034]

In [ ]:
y_val.loc[y_val== 1]

In [ ]:
y_val[23578]

In [ ]:
y_val.sample(25)

In [ ]:
xgbc.fit(X_resampled, y_resampled)

In [ ]:
scores = cross_val_score(xgbc, X_resampled, y_resampled, cv=5)
print("Mean cross-validation score: %.2f" % scores.mean())

 
kfold = KFold(n_splits=10, shuffle=True)
kf_cv_scores = cross_val_score(xgbc, X_resampled, y_resampled, cv=kfold , scoring='roc_auc')
print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

In [ ]:
scores = cross_val_score(xgbc, X_train, y_train, cv=5)
print("Mean cross-validation score: %.2f" % scores.mean())

 
kfold = KFold(n_splits=10, shuffle=True)
kf_cv_scores = cross_val_score(xgbc, X_train, y_train, cv=kfold )
print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

In [ ]:
ypred = xgbc.predict(X_val)
cm = confusion_matrix(y_val,ypred) 
print(cm)


In [ ]:
ypred = xgbc.predict(X_test)
cm = confusion_matrix(y_test,ypred) 
print(cm)

In [ ]:
ypred = xgbc.predict(X_test)


In [ ]:
X_resampled, y_resampled

In [ ]:
cm = confusion_matrix(y_test,ypred) 
print(cm)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
y_pred = xgbc.predict(X_test)
#plt.grid(False)
cm = confusion_matrix(y_test, y_pred)
#plt.grid(False)
cm_display = ConfusionMatrixDisplay(cm).plot()

In [ ]:

y_pred = xgbc.predict(X_test)
#plt.grid(False)
cm = confusion_matrix(y_test, y_pred)
#plt.grid(False)
cm_display = ConfusionMatrixDisplay(cm).plot()

In [ ]:


y_pred = xgbc.predict(X_test)
#plt.grid(False)
cm = confusion_matrix(y_test, y_pred)
#plt.grid(False)
cm_display = ConfusionMatrixDisplay(cm).plot()

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay

In [ ]:
# Import the usual suspects. Any new functions will be introduced individually for clarity.
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC, SVC
from xgboost import XGBClassifier
from collections import Counter

import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.plotting import plot_decision_regions
%matplotlib inline

# make prettier plots
%config InlineBackend.figure_format = 'svg' 

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_sample(X_train,y_train)

In [ ]:
X_resampled.shape

In [ ]:
y_resampled.shape

In [ ]:
Counter(y_resampled)

In [ ]:
cm = print_confusion_matrix(confusion_matrix(y_test, clf_ros.predict(X)), ['Class 0', 'Class 1'])

In [ ]:

y_score = xgbc.decision_function(X_test)

fpr, tpr, _ = roc_curve(y_test, y_score, pos_label=xgbc.classes_[1])
roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr).plot()

In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

In [ ]:
y_train[:10:-1]

In [ ]:
xgbc.fit(X_train, y_train)

y_preds = xgbc.predict_proba(X_test)

In [ ]:
preds = y_preds[:,1]

In [ ]:
preds[:10]

In [ ]:
y_test[:10]

In [ ]:
fpr, tpr, _ = metrics.roc_curve(y_test, preds)

auc_score = metrics.auc(fpr, tpr)

In [ ]:
print(fpr)

In [ ]:
fpr

In [ ]:
plt.clf()

In [ ]:
plt.title('ROC Curve')
plt.plot(fpr, tpr, label='AUC = {:.2f}'.format(auc_score))

In [ ]:
plt.clf()

plt.title('ROC Curve')
plt.plot(fpr, tpr, label='AUC = {:.2f}'.format(auc_score))

# it's helpful to add a diagonal to indicate where chance 
# scores lie (i.e. just flipping a coin)
plt.plot([0,1],[0,1],'r--')

plt.xlim([-0.1,1.1])
plt.ylim([-0.1,1.1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

plt.legend(loc='lower right')
plt.show()

In [ ]:


# load data
#X_train, X_test, y_train, y_test = load_my_data()

# model can be any trained classifier that supports predict_proba()
#clf = LogisticRegression()
#xgbc.fit(X_train, y_train)

#y_preds = xgbc.predict_proba(X_test)

# take the second column because the classifier outputs scores for
# the 0 class as well
#preds = y_preds[:,1]

# fpr means false-positive-rate
# tpr means true-positive-rate
#fpr, tpr, _ = metrics.roc_curve(y_test, preds)

#auc_score = metrics.auc(fpr, tpr)

# clear current figure
plt.clf()

plt.title('ROC Curve')
plt.plot(fpr, tpr, label='AUC = {:.2f}'.format(auc_score))

# it's helpful to add a diagonal to indicate where chance 
# scores lie (i.e. just flipping a coin)
plt.plot([0,1],[0,1],'r--')

plt.xlim([-0.1,1.1])
plt.ylim([-0.1,1.1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

plt.legend(loc='lower right')
plt.show()

In [ ]:
# precision-recall curve and f1 for an imbalanced dataset
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from matplotlib import pyplot

In [ ]:
model = xgbc

In [ ]:
model.fit(X_train, y_train)
# predict probabilities
xgb_probs = model.predict_proba(X_test)
# keep probabilities for the positive outcome only
xgb_probs = xgb_probs[:, 1]

In [ ]:
xgb_probs[:10]

In [ ]:
yhat = model.predict(X_test)

In [ ]:
xgb_precision, xgb_recall, _ = precision_recall_curve(y_test, xgb_probs)

In [ ]:
xgb_f1, xgb_auc = f1_score(y_test, yhat), auc(xgb_recall, xgb_precision)

In [ ]:
print('XGB: f1=%.3f auc=%.3f' % (xgb_f1, xgb_auc))

In [ ]:
no_skill = len(y_test[y_test==1]) / len(y_test)

In [ ]:
pyplot.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
pyplot.plot(xgb_recall, xgb_precision, marker='.', label='XGB')
# axis labels
pyplot.xlabel('Recall')
pyplot.ylabel('Precision')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()

In [ ]:

# generate 2 class dataset
#X, y = make_classification(n_samples=1000, n_classes=2, weights=[0.99,0.01], random_state=1)
# split into train/test sets
#trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2)
# fit a model
#model = LogisticRegression(solver='lbfgs')
model.fit(trainX, trainy)
# predict probabilities
lr_probs = model.predict_proba(testX)
# keep probabilities for the positive outcome only
lr_probs = lr_probs[:, 1]
# predict class values
yhat = model.predict(testX)
# calculate precision and recall for each threshold
lr_precision, lr_recall, _ = precision_recall_curve(testy, lr_probs)
# calculate scores
lr_f1, lr_auc = f1_score(testy, yhat), auc(lr_recall, lr_precision)
# summarize scores
print('Logistic: f1=%.3f auc=%.3f' % (lr_f1, lr_auc))
# plot the precision-recall curves
no_skill = len(y_test[y_test==1]) / len(y_test)
pyplot.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
pyplot.plot(lr_recall, lr_precision, marker='.', label='Logistic')
# axis labels
pyplot.xlabel('Recall')
pyplot.ylabel('Precision')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
# precision-recall curve and f1 for an imbalanced dataset
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from matplotlib import pyplot
# generate 2 class dataset
X, y = make_classification(n_samples=1000, n_classes=2, weights=[0.99,0.01], random_state=1)
# split into train/test sets
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2)
# fit a model
model = LogisticRegression(solver='lbfgs')
model.fit(trainX, trainy)
# predict probabilities
lr_probs = model.predict_proba(testX)
# keep probabilities for the positive outcome only
lr_probs = lr_probs[:, 1]
# predict class values
yhat = model.predict(testX)
# calculate precision and recall for each threshold
lr_precision, lr_recall, _ = precision_recall_curve(testy, lr_probs)
# calculate scores
lr_f1, lr_auc = f1_score(testy, yhat), auc(lr_recall, lr_precision)
# summarize scores
print('Logistic: f1=%.3f auc=%.3f' % (lr_f1, lr_auc))
# plot the precision-recall curves
no_skill = len(testy[testy==1]) / len(testy)
pyplot.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
pyplot.plot(lr_recall, lr_precision, marker='.', label='Logistic')
# axis labels
pyplot.xlabel('Recall')
pyplot.ylabel('Precision')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()

In [ ]:
################ Had missed and need to add back in
fpr, tpr, thresholds = roc_curve(df_eval['in_sf'],
                                 df_eval['proba_sf'])
###########

In [ ]:
def plot_roc(true, probas):
    auc = roc_auc_score(true, probas)

    plt.plot(fpr, tpr, marker='o')
    plt.xlabel('1 - Specificity (FPR)')
    plt.ylabel('Sensitivity (TPR)');
    plt.title(f"Area Under the ROC Curve: {round(auc, 3)}");

In [ ]:
#plot_roc(df_validate['isFraud'], lm_2.predict_proba(X_validate)[:, 1])

In [ ]:
import statsmodels.api as sm

In [ ]:
# This will help with plotting. Look up Pandas.Categorical for more methods ...
logistic_test.loc[:, 'isFraud'] = logistic_test['isFraud'].astype('category')

In [ ]:
test_transaction_identity.info

In [ ]:
df_train


In [ ]:
df_validate

In [ ]:
df_train.loc[:, 'isFraud'] = df_train['isFraud'].astype('category')

In [ ]:
df_validate.loc[:, 'isFraud'] = df_validate['isFraud'].astype('category')

In [ ]:
# For this first example, we'll employ statsmodels
lm_1 = sm.Logit(df_train['isFraud'],  # with statsmodels, `y` comes first
                sm.add_constant(df_train[['hour']]))  # and then `x`
lm_1 = lm_1.fit()

In [ ]:
lm_1.summary()

In [ ]:
# Let's go down the sklearn path, since this is better used in model pipelines
from sklearn.linear_model import LogisticRegression

In [ ]:
# By default, LogisticRegression will fit an intercept, as we did with statsmodels
lm_1 = LogisticRegression(solver='newton-cg',  # For comparison, use the same solver as statsmodels default
                          C=100000)  # No regularization

lm_1.fit(df_train[['hour']], df_train['isFraud'])

In [ ]:
# We can see that the coefficient in question matches the one from statsmodels
print('intercept: ', round(lm_1.intercept_[0], 4))
print('hour coef: ', round(lm_1.coef_[0][0], 4))

In [ ]:
df_validate.info()

In [ ]:
df_eval.info()

In [ ]:
df_eval = df_validate.copy()
df_eval['pred'] = lm_1.predict(df_validate[['hour']])
#df_eval.loc[:, 'pred'] = df_eval['pred'].astype('category')
df_eval['correct_pred'] = df_eval['pred'] == df_eval['isFraud']

In [ ]:
sns.stripplot(data=df_eval,
              x='hour',
              y='pred',
              hue='correct_pred',
              palette={False: '#f03b20', True: '#3182bd'});

In [ ]:
df_column_completion = pd.DataFrame((np.sum(pd.isnull(train_transaction_identity)).
                   sort_values(ascending=False)/len(train_transaction_identity))*100)

In [ ]:
df_column_completion.rename(columns = {0:'% Null'}, inplace = True)

In [ ]:
df_column_completion.head(20
                         )

In [ ]:
train_transaction_identity2 = train_transaction_identity.copy()

In [ ]:
#Remove variables with missing values more than 30 percent

A = (np.sum(pd.isnull(train_transaction_identity2)).sort_values(ascending=False)/len(train_transaction_identity2))*100
Removed_col = A[A>0.3].index
print(Removed_col)
#train_transaction_identity2.drop(columns=Removed_col, inplace=True)

In [ ]:
train_transaction_identity2.shape

In [ ]:
train_transaction_identity2.info()

In [ ]:
train_transaction_identity2.describe()

In [ ]:
train_transaction_identity2.head(25)

In [ ]:
train_transaction_identity2['card6'].value_counts()

In [ ]:
categorical_col_orig = train_transaction_identity.select_dtypes(include=['object']).columns.tolist()
for i in categorical_col_orig: 
    print('% Missing: ',(np.sum(pd.isnull(train_transaction_identity[i]))
        /len(train_transaction_identity[i]))*100)
    print(train_transaction_identity[i].value_counts())
    print('\n')

In [ ]:
np.sum(pd.isnull(train_transaction_identity['card6']))

In [ ]:
for i in categorical_col_orig:
    pyplot.hist((data['colname'])**(1/i),bins=50)
    pyplot.title(f'Transform=1/{i}')
    pyplot.show()

In [ ]:
df = train_transaction_identity

In [ ]:
def plot_features(df, sample_size=1000):
    # We don't need to plot the id (prod and user), and the latest cart is a list
    sample = df['product_id', 'user_id', 'latest_cart'], axis=1) \
               .sample(sample_size, random_state=44)

    g = sns.pairplot(sample, hue='in_cart', plot_kws=dict(alpha=.3, edgecolor='none'))

plot_features(df_X)

In [ ]:
def plot_features(df, sample_size=1000):
    # We don't need to plot the id (prod and user), and the latest cart is a list
    sample = df.drop(['product_id', 'user_id', 'latest_cart'], axis=1) \
               .sample(sample_size, random_state=44)

    g = sns.pairplot(sample, hue='in_cart', plot_kws=dict(alpha=.3, edgecolor='none'))

plot_features(df_X)

In [ ]:
1571/590540

In [ ]:
(np.sum(pd.isnull(train_transaction_identity['card6']))
/len(train_transaction_identity['card6']))*100

In [ ]:

for i in ['isFraud','card6']: 
    print('% Missing: ',(np.sum(pd.isnull(df_train[i]))
        /len(df_train[i]))*100)
    print(df_train[i].value_counts())
    print('\n')

In [ ]:

for i in ['isFraud','card6']: 
    print('% Missing: ',(np.sum(pd.isnull(df_validate[i]))
        /len(df_validate[i]))*100)
    print(df_validate[i].value_counts())
    print('\n')

In [ ]:
categorical_col = train_transaction_identity2.select_dtypes(include=['object']).columns.tolist()
for i in categorical_col: 
    print('% Missing: ',(np.sum(pd.isnull(train_transaction_identity[i]))
        /len(train_transaction_identity[i]))*100)
    print(train_transaction_identity2[i].value_counts())
    print('\n')

In [ ]:
categorical_col = train_transaction_identity2.select_dtypes(include=['object']).columns.tolist()
for i in categorical_col: 
    print('% Missing: ',(np.sum(pd.isnull(train_transaction_identity[i]))
        /len(train_transaction_identity[i]))*100)
    print(train_transaction_identity2[i].value_counts())
    print('\n')

In [ ]:
dataframe_dtypes = pd.DataFrame(train_transaction_identity2.dtypes)

In [ ]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 50)

In [ ]:
dataframe_dtypes

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)

In [ ]:
train_dummy = pd.get_dummies(train_transaction_identity2, drop_first=True)
train_dummy.head()